<a href="https://colab.research.google.com/github/aim56009/AI-MA_project/blob/main/Chew.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import pickle
import torchvision.transforms.functional as TF 

from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
import glob
import os

import random
import click
import sklearn
import sklearn.model_selection
from collections import defaultdict
from sklearn.metrics import accuracy_score
from pathlib import Path
import sys
from torch import optim
from torch.optim import lr_scheduler
from functools import partial
import matplotlib.pyplot as plt

In [2]:
!git clone https://github.com/aim56009/AI-MA_project.git

fatal: destination path 'AI-MA_project' already exists and is not an empty directory.


#Dataloader

In [3]:
batch_size = 1 
PATH_TO_DATA = "AI-MA_project/pianoroll"
workers = 0

In [4]:
class MusicDataset(Dataset):

    def __init__(self, data_dir, transforms=None):
        self.transforms = transforms
        self.data_dir = data_dir

        labels = ["voice_0", "voice_1", "voice_2", "voice_3", "voice_all"]
        self.labels = labels
        self.pr_dict = {}
        len_list = []

        for iLabel in range(len(labels)):
            
            if iLabel == 4:   
                voice_files = []
                file_names = sorted(glob.glob(os.path.join(PATH_TO_DATA, self.labels[iLabel], "*.pkl")))   
                for name in file_names:
                    with open(name ,'rb') as f: 
                        loaded_obj = pickle.load(f)  
                        voice_files.append(loaded_obj) 
                        len_list.append(len(loaded_obj.T))                             
                self.pr_dict[self.labels[iLabel]] = voice_files 
                self.pr_dict["length"] = len_list
            
            else:
                voice_files = []
                file_names = sorted(glob.glob(os.path.join(PATH_TO_DATA, self.labels[iLabel], "*.pkl")))   
                for name in file_names:
                    with open(name ,'rb') as f: 
                        loaded_obj = pickle.load(f)     
                        voice_files.append(loaded_obj)                              
                self.pr_dict[self.labels[iLabel]] = voice_files 
                                        

    def __len__(self):
      
        return len(self.pr_dict[self.labels[0]])

    def __getitem__(self, idx):          

        out_list = []
        for key, value in self.pr_dict.items():
            out_list.append(self.pr_dict[key][idx])                  
        v0 = torch.tensor(out_list[0].T)
        v1 = torch.tensor(out_list[1].T)
        v2 = torch.tensor(out_list[2].T)
        v3 = torch.tensor(out_list[3].T)
        v_all = torch.tensor(out_list[4].T) 
        length = self.pr_dict["length"][idx]

        return (v0, v1, v2, v3, v_all, length)

In [5]:
dataset = MusicDataset(PATH_TO_DATA)
loader = torch.utils.data.DataLoader(dataset,batch_size=batch_size, shuffle=False, num_workers=workers, drop_last=True)

In [6]:
dataset[326][0].shape

torch.Size([576, 128])

In [7]:
"""
0 = next(iter(loader))[0]
v1 = next(iter(loader))[1]
v2 = next(iter(loader))[2]
v3 = next(iter(loader))[3]
v4 = next(iter(loader))[4]
label = next(iter(loader))[5]

voices = next(iter(loader))[0:5]
"""

'\n0 = next(iter(loader))[0]\nv1 = next(iter(loader))[1]\nv2 = next(iter(loader))[2]\nv3 = next(iter(loader))[3]\nv4 = next(iter(loader))[4]\nlabel = next(iter(loader))[5]\n\nvoices = next(iter(loader))[0:5]\n'

# Chew


In [8]:
!pip3 install partitura
import partitura
import numpy as np
import os
import pickle

In [12]:
name = "chor133.xml"


import_path = os.path.join("AI-MA_project/chorales_converted", name)
part = partitura.load_musicxml(import_path)

OSError: ignored

In [10]:
path_xml = "AI-MA_project/chorales_converted"

for filename in os.listdir(path_xml):
    fullname = os.path.join(path, filename)
    part = partitura.load_musicxml(fullname)
        
    part_zero = part[0]
    part_one = part[1]
    part_two = part[2]
    part_three = part[3]

    part_zero

FileNotFoundError: ignored

In [ ]:
path = "AI-MA_project/chorales_converted"

for filename in os.listdir(path):
    if not filename.endswith('.xml'): continue
    fullname = os.path.join(path, filename)
    part = partitura.load_musicxml(fullname)
        
    part_zero = part[0]
    part_one = part[1]
    part_two = part[2]
    part_three = part[3]

    part_zero

In [ ]:
path = "/Users/sayang/Masterproject/chorales_converted"

for filename in os.listdir(path):
    if not filename.endswith('.xml'): continue
    fullname = os.path.join(path, filename)
    part = partitura.load_musicxml(fullname)
        
    part_zero = part[0]
    part_one = part[1]
    part_two = part[2]
    part_three = part[3]
        
    
    pr_zero = partitura.utils.compute_pianoroll(part_zero, time_unit = "beat",time_div = 12)
    pr_zero = pr_zero.toarray()

    pr_one = partitura.utils.compute_pianoroll(part_one, time_unit = "beat",time_div = 12)
    pr_one = pr_one.toarray()

    pr_two = partitura.utils.compute_pianoroll(part_two, time_unit = "beat",time_div = 12)
    pr_two = pr_two.toarray()

    pr_three = partitura.utils.compute_pianoroll(part_three, time_unit = "beat",time_div = 12)
    pr_three = pr_three.toarray()

    pr = partitura.utils.compute_pianoroll(part, time_unit = "beat",time_div = 12)
    pr = pr.toarray()
    
    if pr_zero.shape == pr_one.shape == pr_two.shape == pr_three.shape == pr.shape : 
        pr_comp = pr_zero+pr_one+pr_two+pr_three
        pr_comp =  np.where(pr_comp < 1 , pr_comp, 1) 

        file_out_names = [ "voice_0_" + filename[4:7]+ ".pkl" , "voice_1_" + filename[4:7] + ".pkl" , "voice_2_" + filename[4:7] + ".pkl", "voice_3_" + filename[4:7] + ".pkl", "voice_all_" + filename[4:7]+".pkl"   ]    
        voice_folder = ["voice_0","voice_1","voice_2","voice_3","voice_all"]
        pianorolls = [pr_zero,pr_one,pr_two,pr_three,pr]


        for n,i in enumerate(pianorolls):
            a_folder = voice_folder[n]
            a_file = file_out_names[n]
            joined_path = os.path.join("pianoroll", a_folder, a_file)
            #if a_file not in  ["chor133.xml", "chor043.xml", "chor283.xml"]:
            with open(joined_path, 'wb') as f:
                pickle.dump(i, f)    
                
    else:
        print(filename)


In [ ]:
name = "chor326.xml"
part = partitura.load_musicxml(name)

In [ ]:
chew_sep = partitura.musicanalysis.estimate_voices(part, monophonic_voices=True)
#chew_sep_

In [ ]:
pos_zero = np.where(chew_sep==1)
pos_one = np.where(chew_sep==2)
pos_two = np.where(chew_sep==3)
pos_three = np.where(chew_sep==4)


pos_zero,pos_one,pos_two,pos_three

In [ ]:
note_array_zero = partitura.utils.ensure_notearray(part)[pos_zero]
note_array_one = partitura.utils.ensure_notearray(part)[pos_one]
note_array_two = partitura.utils.ensure_notearray(part)[pos_two]
note_array_three = partitura.utils.ensure_notearray(part)[pos_three]

In [ ]:
pr_zero = partitura.utils.compute_pianoroll(note_array_zero,time_unit = "beat",time_div = 12)
pr_zero = pr_zero.toarray()

pr_one = partitura.utils.compute_pianoroll(note_array_one,time_unit = "beat",time_div = 12)
pr_one = pr_one.toarray()

pr_two = partitura.utils.compute_pianoroll(note_array_two,time_unit = "beat",time_div = 12)
pr_two = pr_two.toarray()

pr_three = partitura.utils.compute_pianoroll(note_array_three,time_unit = "beat",time_div = 12)
pr_three = pr_three.toarray()


In [ ]:
part_zero = part[0]
part_one = part[1]
part_two = part[2]
part_three = part[3]

In [ ]:
pr_zero_ori = partitura.utils.compute_pianoroll(part_zero, time_unit = "beat",time_div = 12)
v0 = pr_zero_ori.toarray()

pr_one_ori = partitura.utils.compute_pianoroll(part_one, time_unit = "beat",time_div = 12)
v1 = pr_one_ori.toarray()

pr_two_ori = partitura.utils.compute_pianoroll(part_two, time_unit = "beat",time_div = 12)
v2 = pr_two_ori.toarray()

pr_three_ori = partitura.utils.compute_pianoroll(part_three, time_unit = "beat",time_div = 12)
v3 = pr_three_ori.toarray()

pr_ori = partitura.utils.compute_pianoroll(part, time_unit = "beat",time_div = 12)
v_all = pr_ori.toarray()

In [ ]:
#pr_zero_ori.shape, pr_one_ori.shape,pr_two_ori.shape, pr_three_ori.shape, pr_ori.shape
#pr_zero.shape, pr_one.shape,pr_two.shape, pr_three.shape

In [ ]:
#np.sum(pr_zero_ == pr_zero) - 61440, np.sum(pr_one == pr_one_) - 61440 ,np.sum(pr_two == pr_two_) - 61440,np.sum(pr_three_ == pr_three) - 61440

In [ ]:
loss = nn.CrossEntropyLoss(reduction="mean")
"""
pr_zero = torch.tensor(pr_zero).type(torch.FloatTensor)
v0 = torch.tensor(v0).type(torch.FloatTensor)

pr_one = torch.tensor(pr_one).type(torch.FloatTensor)
v1 = torch.tensor(v1).type(torch.FloatTensor)

pr_two = torch.tensor(pr_two).type(torch.FloatTensor)
v2 = torch.tensor(v2).type(torch.FloatTensor)

pr_three = torch.tensor(pr_three).type(torch.FloatTensor)
v3 = torch.tensor(v3).type(torch.FloatTensor)

v0 = v0.squeeze()
v1 = v1.squeeze()
v2 = v2.squeeze()
v3 = v3.squeeze()
        
losses = loss(pr_zero, v0) + loss(pr_one, v1) + loss(pr_two, v2) + loss(pr_three, v3)
losses
"""

In [ ]:
from sklearn.metrics import accuracy_score

acc_v0 = accuracy_score(torch.tensor(pr_zero), v0)
acc_v1 = accuracy_score(torch.tensor(pr_one), v1)
acc_v2 = accuracy_score(torch.tensor(pr_two),v2)
acc_v3 = accuracy_score(torch.tensor(pr_three), v3)

In [ ]:
acc_v0,acc_v1,acc_v2,acc_v3